# Day 10: Multiobjective Optimization

---

Optimization is considerably more complex when we have multiple, competing objectives.

- Examples with multiple objectives
- Concepts:
  - Dominance, the Pareto frontier
  - Scalarization
  - Making choices


In [1]:
import pandas as pd 
import grama as gr 
import plotnine as pt 
DF = gr.Intention()


## Examples with multiple objectives

---


### Buildings

$$\min\, \text{(Weight of building)}$$
$$\min\, \text{(Cost of building)}$$



$$\min\, \text{(Safety of building)}$$
$$\min\, \text{(Cost of building)}$$


### Communications

$$\min\, \text{(Error rate)}$$
$$\min\, \text{(Data rate)}$$


### Materials

$$\min\, \text{Strength}$$
$$\min\, \text{Toughness}$$

## Dominance and the Pareto Frontier

---


## Scalarization

---


## Tips on Making Choices

---
